# PyGeM

###### Tutorial 7: Free Form Deformation on a Kratos mdpa mesh on cylinder: continuity of the deformation

In this tutorial we show how to perform the Free Form Deformation on a hexaedral mesh of a cylinder. As seen in Tutorial 3, but with a Kratos mdpa file.

As PyGeM cannot visualize mdpa files directly, meshio and vedo are needed to convert the file to a vtk file and then plot it. The library vedo allows for interactive plot within Jupyter. See below.  These can be installed with pip install vedo meshio.

First of all we just import the required PyGeM classes and we read a parameters file.

In [8]:
import pygem as pg
from pygem.mdpahandler import MdpaHandler
import vedo
import meshio

The following is the parameters file for the case at hand. In particular, if you look at the Box info section, there is 2-by-2-by-3 lattice around a cylinder. Since we want to shift the middle section of the cylinder along the x direction we modify only the parameter x weights corresponding to the middle point in z direction (index==1). In the following we show only the important parts of the parameters file

In [9]:
ffd = pg.FFD()
ffd.read_parameters('../tests/test_datasets/parameters_test_ffd_pipe_unv_C0.prm')

In [10]:
%cat ../tests/test_datasets/parameters_test_ffd_pipe_unv_C0.prm


[Box info]
# This section collects all the properties of the FFD bounding box.

# n control points indicates the number of control points in each direction (x, y, z).
# For example, to create a 2 x 3 x 2 grid, use the following: n control points: 2, 3, 2
n control points x: 2
n control points y: 2
n control points z: 3

# box length indicates the length of the FFD bounding box along the three canonical directions (x, y, z).
# It uses the local coordinate system.
# For example to create a 2 x 1.5 x 3 meters box use the following: box length: 2.0, 1.5, 3.0
box length x: 2.2
box length y: 2.2
box length z: 6.0

# box origin indicates the x, y, and z coordinates of the origin of the FFD bounding box. That is center of
# rotation of the bounding box. It corresponds to the point coordinates with position [0][0][0].
# See section "Parameters weights" for more details.
# For example, if the origin is equal to 0., 0., 0., use the following: box origin: 0., 0., 0.
box origin x: -1.1
box origin 

We now load the mdpa file!

In [16]:
mdpa_handler = MdpaHandler()
mesh_points = mdpa_handler.parse('../tests/test_datasets/test_pipe.mdpa')

and visualize the undeformed mesh by tranlating with meshio from MPDA to VTK format.

In [17]:
UndeformedMDPA = meshio.read('../tests/test_datasets/test_pipe.mdpa')
meshio.write('test_pipe_undeformed_mdpa.vtk', UndeformedMDPA)
UndeformedVTK = vedo.load('test_pipe_undeformed_mdpa.vtk')
vedo.settings.embedWindow(backend='k3d', verbose=True)
vedo.show(UndeformedVTK, viewup="z", resetcam=True)

Plot(antialias=3, axes=['x', 'y', 'z'], axes_helper=1.0, background_color=16777215, camera=[5.469634129164876,…

We now perform the FFD and write out the results with a new mdpa file.

In [18]:
new_mesh_points = ffd(mesh_points)
mdpa_handler.write(new_mesh_points, 'test_pipe_mod_C0.mdpa')

Let us see the result.

In [19]:
DeformedMDPA = meshio.read('test_pipe_mod_C0.mdpa')
meshio.write('test_pipe_deformed_mdpa.vtk', DeformedMDPA)
DeformedVTK = vedo.load('test_pipe_deformed_mdpa.vtk')
vedo.settings.embedWindow(backend='k3d', verbose=True)
vedo.show(DeformedVTK, viewup="z", resetcam=True)

Plot(antialias=3, axes=['x', 'y', 'z'], axes_helper=1.0, background_color=16777215, camera=[6.017318527774512,…

As you can easily see the deformation makes the mesh only C0 continuous.

This is not wrong a priori, but it can have some drawbacks.

First of all, if you start with a smooth geometry, probably you do not want to end up with a geometry with more edges.

For large deformations this can cause inaccurancy errors when you try to perform analysis on the deformed mesh. In fact, it can happen that we have very stretched cells that the solver can not treat properly. Of course it depends mainly on the solver choosen, but, in general, it is not very clever to have "spiky" cells.

Moreover, if you are interested in some quantities, such as grandients and normal vectors, it can be difficult to compute them close to the "artificial edges".

Thus, we can rely on the properties of the Bernstein polynomials (on which FFD is built upon) to overcome this potential problem. In fact, if we add 2 new control points in the z direction we can move only the middle one, leaving the first two and the last two still. The new parameter file becomes:

In [ ]:
%cat ../tests/test_datasets/parameters_test_ffd_pipe_unv_C1.prm

We again load the new parameter file, perform the FFD and write out the results with a new mdpa file.

In [ ]:
ffd = pg.FFD()
ffd.read_parameters('../tests/test_datasets/parameters_test_ffd_pipe_unv_C1.prm')
new_mesh_points = ffd(mesh_points)
mdpa_handler.write(new_mesh_points, 'test_pipe_mod_C1.mdpa')

And here it is the C1 mesh on the cylinder.

In [ ]:
mesh = meshio.read('test_pipe_mod_C1.mdpa')
meshio.write('test_pipe_mod_C1_mdpa.vtk', mesh)

In [ ]:
DeformedMDPA = meshio.read('test_pipe_mod_C1.mdpa')
meshio.write('test_pipe_deformed_mdpa.vtk', DeformedMDPA)
DeformedVTK = vedo.load('test_pipe_deformed_mdpa.vtk')
vedo.settings.embedWindow(backend='k3d', verbose=True)
vedo.show(DeformedVTK, viewup="z", resetcam=True)

You can add further points to increase again the continuity of the mesh!